# Project : Product Return Prediction

## Introduction
In this project, we focus on understanding the reasons behind product returns and predict whether a product will be returned. Our primary goal is to provide actionable insights to retailers rather than achieving the maximum model accuracy. By identifying patterns and factors that leads to returns, we aim to help businesses to reduce return rates and associated costs.

## Problem Statement
To Predict the likelihood of a product being returned based on various features such as product category, price and customer demographics. The goal of this project is to provide insights into the factors that contribute to product returns, enabling reatilers to make informed decisions and improve their return policies and customer experience.

## Business Problem
In the retail industry, product returns are a significant challenge, impacting profitablity and customer satisfaction. Understanding the reasons behind returns can help retailers improve their products, services and customer experience.     

## Mapping the Real-World Problem to a Machine Learning Problem
**It is a machine learning binary classification problem where we predict whether a order will be returned or not. By analyzing various order features, its aim to help businesses reduce return rates and improve customer satisfaction.**

### Import Libraries

In [23]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt 
import sklearn
from scipy import stats
import kaggle
from pathlib import Path
import sidetable

### Dispaly settings

In [38]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
#pd.set_option('display.max_info_rows', 1000)
pd.set_option('display.float_format', '{:.2f}'.format)
import warnings
warnings.filterwarnings('ignore')


### Data Ingestion and Exploration

In [25]:
path = Path.cwd()
paths = path.parent.joinpath('data','raw')

file = []
for p in paths.glob('*'):
    file.append(p.name)

print(file)

['.gitkeep', 'Global Superstore.xls']


In [26]:
#find all excel files name in .xls sheet
df = pd.read_excel(paths/'Global Superstore.xls',sheet_name=None)
df.keys()

dict_keys(['Orders', 'Returns', 'People'])

In [27]:
order_df = pd.read_excel(paths/'Global Superstore.xls',sheet_name='Orders')
return_df = pd.read_excel(paths/'Global Superstore.xls',sheet_name='Returns')
people_df = pd.read_excel(paths/'Global Superstore.xls',sheet_name='People')
display(order_df.tail(1),
    return_df.tail(1),
    people_df.tail(1))

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,Country,Postal Code,Market,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
51289,6147,MX-2012-134460,2012-05-22,2012-05-26,Second Class,MC-18100,Mick Crebagga,Consumer,Tipitapa,Managua,Nicaragua,NaN,LATAM,Central,OFF-PA-10004155,Office Supplies,Paper,"Eaton Computer Printout Paper, 8.5 x 11",61.38,3,0.00,1.80,0.00,High


,Returned,Order ID,Market
1172,Yes,CA-2014-168193,United States


,Person,Region
12,Alejandro Ballentine,Southeast Asia


In [28]:
display('Orders shape :',order_df.shape, 
        'Returns shape', return_df.shape, 
        'People shape',people_df.shape)

'Orders shape :'

(51290, 24)

'Returns shape'

(1173, 3)

'People shape'

(13, 2)

In [29]:
# remove whitespaces from column names
order_df.columns = order_df.columns.str.strip()
return_df.columns = return_df.columns.str.strip()
people_df.columns = people_df.columns.str.strip()

In [30]:
# Lets lower case all column names
order_df.columns = map(lambda col: col.lower(), order_df.columns)
return_df.columns = map(lambda col: col.lower(), return_df.columns)
people_df.columns = map(lambda col: col.lower(), people_df.columns)

In [31]:
order_df.columns

Index(['row id', 'order id', 'order date', 'ship date', 'ship mode',
       'customer id', 'customer name', 'segment', 'city', 'state', 'country',
       'postal code', 'market', 'region', 'product id', 'category',
       'sub-category', 'product name', 'sales', 'quantity', 'discount',
       'profit', 'shipping cost', 'order priority'],
      dtype='object')

In [40]:
# lets check the data types of the columns
order_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51290 entries, 0 to 51289
Data columns (total 24 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   row id          51290 non-null  int64         
 1   order id        51290 non-null  object        
 2   order date      51290 non-null  datetime64[ns]
 3   ship date       51290 non-null  datetime64[ns]
 4   ship mode       51290 non-null  object        
 5   customer id     51290 non-null  object        
 6   customer name   51290 non-null  object        
 7   segment         51290 non-null  object        
 8   city            51290 non-null  object        
 9   state           51290 non-null  object        
 10  country         51290 non-null  object        
 11  postal code     9994 non-null   float64       
 12  market          51290 non-null  object        
 13  region          51290 non-null  object        
 14  product id      51290 non-null  object        
 15  ca

In [42]:
return_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1173 entries, 0 to 1172
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   returned  1173 non-null   object
 1   order id  1173 non-null   object
 2   market    1173 non-null   object
dtypes: object(3)
memory usage: 27.6+ KB


In [43]:
people_df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13 entries, 0 to 12
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   person  13 non-null     object
 1   region  13 non-null     object
dtypes: object(2)
memory usage: 340.0+ bytes


**Dataset Summary**

**The Global Superstore dataset contains three sheets: Orders, Returns and People.**
    
- **The Orders sheet contains information about customer orders, including order ID, customer ID, demographic details ,product details and sales data.**    
- **The Returns sheet contains information about returned orders, including order ID and market details.**    
- **The people sheet contains information about customers, including customer name and demographic details.**
    
**The data spans a robust four-year period from 2011 to 2015 . This dataset contains a vast array of order-related information, offering insights into retail operations across 147 countries.**


In [59]:
order_df.columns

Index(['row id', 'order id', 'order date', 'ship date', 'ship mode',
       'customer id', 'customer name', 'segment', 'city', 'state', 'country',
       'postal code', 'market', 'region', 'product id', 'category',
       'sub-category', 'product name', 'sales', 'quantity', 'discount',
       'profit', 'shipping cost', 'order priority'],
      dtype='object')

Brief description of columns:

- **Order ID**: Unique identifier for each order.

- **Order Date**: The date when the order was placed. 

- **Ship Date**: The date when the order was shipped. 

- **Ship Mode**: The day order was shipped (e.g., First Class, Second Class, Standard Class, Same Day).

- **Customer ID**: Unique identifier for each customer. 

- **Customer Name**: The name of the customer. 

- **Segment**: The business segment (e.g., Consumer, Corporate, Home Office). 

- **City**: The city where the order was placed.

- **State**: The state where the order was placed.

- **Country**: The country where the order was placed.

- **Postal Code**: The postal code of the order's location.

- **Market**: The region where the sale took place (e.g., APAC, EMEA). 

- **Region**: The specific geographical region (e.g., Central, East). 

- **Product ID**: Unique identifier for each product.

- **Product Category**: The category of products (e.g., Furniture, Technology, Office Supplies). 

- **Product Sub-Category**: More specific product grouping (e.g., Chairs, Phones). 

- **Sales**: The total sales value of the order. 

- **Quantity**: Quantity of products sold. 

- **Discount**: Discount applied to the product. 

- **Profit**: The profit earned from the order. 

- **Shipping Cost**: The cost incurred to ship the product.

